In [26]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut

In [3]:
def knn(df):
    data=df.to_numpy()
    X=data[:,:-1]
    y=data[:,-1]
    loo = LeaveOneOut()
    ture_preds=0
    miss_classified_index=[]
    all_y=[]
    all_probs=[]
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        scaler = StandardScaler()
        X_train=scaler.fit_transform(X_train)
        X_test=scaler.transform(X_test)
        clf=KNeighborsClassifier()
        clf.fit(X_train, y_train)
        all_y.append(y_test)
        all_probs.append(clf.predict_proba(X_test)[:,1])
        if clf.predict(X_test)!=y_test:
            miss_classified_index.append(test_index[0].item())
        ture_preds+=clf.predict(X_test)==y_test
    acc=ture_preds[0]/loo.get_n_splits(X)*100
    print("Accuracy KNN:",acc)
    rate_wt=0
    rate_tg=0
    for x in names[miss_classified_index]:
        if x[:2]=='WT':
            rate_wt+=1
        else:
            rate_tg+=1
    all_y = np.array(all_y)
    all_probs = np.array(all_probs)
    print("Accuracy WT:",abs((rate_wt/(df.label==0).sum()*100)-100))
    print("Accuracy TG:",abs((rate_tg/(df.label==1).sum()*100)-100))
    return(acc)

In [4]:
def svm(df):
    data=df.to_numpy()
    X=data[:,:-1]
    y=data[:,-1]
    loo = LeaveOneOut()
    ture_preds=0
    miss_classified_index=[]
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf = make_pipeline(StandardScaler(),
                        LinearSVC(random_state=0))
        clf.fit(X_train, y_train)
        if clf.predict(X_test)!=y_test:
            miss_classified_index.append(test_index[0].item())
        ture_preds+=clf.predict(X_test)==y_test
    acc=ture_preds[0]/loo.get_n_splits(X)*100
    print("Accuracy SVM:",ture_preds[0]/loo.get_n_splits(X)*100)
    rate_wt=0
    rate_tg=0
    for x in names[miss_classified_index]:
        if x[:2]=='WT':
            rate_wt+=1
        else:
            rate_tg+=1
    print("Accuracy WT:",abs((rate_wt/(df.label==0).sum()*100)-100))
    print("Accuracy TG:",abs((rate_tg/(df.label==1).sum()*100)-100))
    return acc

In [5]:
def rf(df):
    data=df.to_numpy()
    X=data[:,:-1]
    y=data[:,-1]
    loo = LeaveOneOut()
    ture_preds=0
    miss_classified_index=[]
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf = make_pipeline(RandomForestClassifier(criterion='entropy',random_state=0))
        clf.fit(X_train, y_train)
        if clf.predict(X_test)!=y_test:
            miss_classified_index.append(test_index[0].item())
        ture_preds+=clf.predict(X_test)==y_test
    acc=ture_preds[0]/loo.get_n_splits(X)*100
    print("Accuracy RF:",acc)
    rate_wt=0
    rate_tg=0
    for x in names[miss_classified_index]:
        if x[:2]=='WT':
            rate_wt+=1
        else:
            rate_tg+=1
    print("Accuracy WT:",abs((rate_wt/(df.label==0).sum()*100)-100))
    print("Accuracy TG:",abs((rate_tg/(df.label==1).sum()*100)-100))
    return acc

In [6]:
# Loading mean PSD of each mouse
wt=pd.read_csv('D:/Projects/SpikeSorting/matlab_files/wt_final.csv')
tg=pd.read_csv('D:/Projects/SpikeSorting/matlab_files/tg_final.csv')

In [7]:
wt['label']=0
tg['label']=1

In [8]:
df=pd.concat([wt,tg],axis=0,ignore_index=True)
names=df['name']
df.drop('name',axis=1,inplace=True)

# Classification

## KNN

In [31]:
print('Input features: '+', '.join(df.columns[0:4]))
knn(df.iloc[:,[0,1,2,3,4]])
print(10*'-')
print('Input features: '+', '.join(df.columns[0:3]))
knn(df.iloc[:,[0,1,2,4]])
print(10*'-')
print('Input features: '+', '.join(df.columns[0:2]))
knn(df.iloc[:,[0,1,4]])
print(10*'-')
print('Input features: '+', '.join(df.columns[1:2]))
knn(df.iloc[:,[1,4]])

Input features: Delta, Theta, Slow Gamma, Fast Gamma
Accuracy KNN: 83.33333333333334
Accuracy WT: 85.71428571428572
Accuracy TG: 81.81818181818181
----------
Input features: Delta, Theta, Slow Gamma
Accuracy KNN: 66.66666666666666
Accuracy WT: 57.142857142857146
Accuracy TG: 72.72727272727273
----------
Input features: Delta, Theta
Accuracy KNN: 61.111111111111114
Accuracy WT: 57.142857142857146
Accuracy TG: 63.63636363636363
----------
Input features: Theta
Accuracy KNN: 50.0
Accuracy WT: 14.285714285714292
Accuracy TG: 72.72727272727273


50.0

## SVM

In [32]:
print('Input features: '+', '.join(df.columns[0:4]))
svm(df.iloc[:,[0,1,2,3,4]])
print(10*'-')
print('Input features: '+', '.join(df.columns[0:3]))
svm(df.iloc[:,[0,1,2,4]])
print(10*'-')
print('Input features: '+', '.join(df.columns[0:2]))
svm(df.iloc[:,[0,1,4]])
print(10*'-')
print('Input features: '+', '.join(df.columns[1:2]))
svm(df.iloc[:,[1,4]])

Input features: Delta, Theta, Slow Gamma, Fast Gamma
Accuracy SVM: 83.33333333333334
Accuracy WT: 85.71428571428572
Accuracy TG: 81.81818181818181
----------
Input features: Delta, Theta, Slow Gamma
Accuracy SVM: 72.22222222222221
Accuracy WT: 71.42857142857143
Accuracy TG: 72.72727272727273
----------
Input features: Delta, Theta
Accuracy SVM: 44.44444444444444
Accuracy WT: 14.285714285714292
Accuracy TG: 63.63636363636363
----------
Input features: Theta
Accuracy SVM: 55.55555555555556
Accuracy WT: 0.0
Accuracy TG: 90.9090909090909


55.55555555555556

## RF

In [33]:
print('Input features: '+', '.join(df.columns[0:4]))
rf(df.iloc[:,[0,1,2,3,4]])
print(10*'-')
print('Input features: '+', '.join(df.columns[0:3]))
rf(df.iloc[:,[0,1,2,4]])
print(10*'-')
print('Input features: '+', '.join(df.columns[0:2]))
rf(df.iloc[:,[0,1,4]])
print(10*'-')
print('Input features: '+', '.join(df.columns[1:2]))
rf(df.iloc[:,[1,4]])

Input features: Delta, Theta, Slow Gamma, Fast Gamma
Accuracy RF: 94.44444444444444
Accuracy WT: 100.0
Accuracy TG: 90.9090909090909
----------
Input features: Delta, Theta, Slow Gamma
Accuracy RF: 77.77777777777779
Accuracy WT: 71.42857142857143
Accuracy TG: 81.81818181818181
----------
Input features: Delta, Theta
Accuracy RF: 55.55555555555556
Accuracy WT: 42.85714285714286
Accuracy TG: 63.63636363636363
----------
Input features: Theta
Accuracy RF: 50.0
Accuracy WT: 42.85714285714286
Accuracy TG: 54.54545454545455


50.0